In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import csv
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from ipywidgets import interact, FloatSlider
from IPython.display import display
%matplotlib inline

Load and fit Laughlin L2 data

In [2]:
L2_fit = []
with open('D:\\folders\\fly_motion_detection\lum_con_integration project\Madhura_codes\contrastResponses.csv', newline = '') as csvfile:
    data = csv.reader(csvfile)
    for row in data:
        L2_fit.append(np.array(row, dtype='float'))
L2_fit = np.array(L2_fit)

In [3]:
def sigmoid(x, a, b, c):
    return a/(1+np.exp(-b*(x-c)))

In [4]:
popt = curve_fit(sigmoid, L2_fit[:, 0], L2_fit[:, 1], bounds=(0, [200., 2., 10.]))

In [5]:
maxL = 1176758.88 #maximal luminance

ND_mixed = np.array([1, 8, 64, 512, 4096])

backL_mixed = np.array([2, 3, 4, 5, 6, 8])

edgeL_mixed = np.array([1, 1, 1, 1, 1, 1])

lumValues_mixed = ((1/ND_mixed).reshape([len(ND_mixed), 1]) @ backL_mixed.reshape([1, len(backL_mixed)]))/15 * maxL

lum_edge_mixed = ((1/ND_mixed).reshape([len(ND_mixed), 1]) @ edgeL_mixed.reshape([1, len(edgeL_mixed)]))/15 * maxL

Adaptlum_mixed = (np.mean(lumValues_mixed, axis=1)*(0.5 + 0.75/2) + np.mean(lum_edge_mixed, axis=1)*0.75/2)/(0.5 + 0.75 + 1)

data_dir = '../../lum_con_integration project 45/'

Mixed contrast control

In [6]:
data1_ND = ['0.0', '0.9', '1.8', '2.7', '3.6']
int_turn_mixed_1 = np.zeros(lumValues_mixed.shape)
luminance_mixed = np.copy(lumValues_mixed)

for i in range(len(data1_ND)):
    data1 = scipy.io.loadmat(data_dir+'mixed_contrasts_Off_int_turn/UAS-shi_1toOFF_int0_192dps_96amps_ND'\
                             +data1_ND[i]+'.mat')
    int_turn_mixed_1[i, :] = data1['groupMean']

Mixed contrast L3 silenced

In [7]:
int_turn_mixed_2 = np.zeros(lumValues_mixed.shape)

for i in range(len(data1_ND)):
    data2 = scipy.io.loadmat(data_dir+'mixed_contrasts_Off_int_turn/L3[0595]-shi_1toOFF_int0_192dps_96amps_ND'\
                             +data1_ND[i]+'.mat')
    int_turn_mixed_2[i, :] = data2['groupMean']

In [8]:
L3_active_all = np.concatenate((np.ones(lumValues_mixed.size), np.zeros(lumValues_mixed.size)))

lum_edge_all = np.concatenate((lum_edge_mixed.flatten('F'), lum_edge_mixed.flatten('F')))

real_response_all = np.concatenate((int_turn_mixed_1.flatten('F'), int_turn_mixed_2.flatten('F')))

Defining fitting function

In [9]:
log10_Im = np.log10(612.895)
kI = 10

In [10]:
def fitting_mixed(params):
    
    k0 = params[0]
    idx_test = params[1]
    idx_train = np.setdiff1d(np.arange(60), idx_test)
    
    popt_new = np.copy(popt[0])
    
    contrast_mixed = np.zeros(lumValues_mixed.shape)
    L2_response_mixed = np.zeros(lumValues_mixed.shape)
    
    for i in range(len(data1_ND)):
        contrast_mixed[i, :] = (lum_edge_mixed[i,:] - lumValues_mixed[i, :])/Adaptlum_mixed[i]
        k1 = k0 * sigmoid(np.log10(Adaptlum_mixed[i]), *popt_new)/popt_new[0]
        L2_response_mixed[i, :] = np.tanh(-contrast_mixed[i, :]*k1)
        
    filter_high_mixed = sigmoid(np.log10(luminance_mixed), 1, kI, log10_Im)
    L3_response_high_mixed = L2_response_mixed * filter_high_mixed
    L3_response_low_mixed = np.log10(luminance_mixed) * (1 - filter_high_mixed)
    
    L2_response_all = np.concatenate((L2_response_mixed.flatten('F'), L2_response_mixed.flatten('F')))

    filter_high_all = np.concatenate((filter_high_mixed.flatten('F'), filter_high_mixed.flatten('F')))

    L3_response_high_all = np.concatenate((L3_response_high_mixed.flatten('F'), L3_response_high_mixed.flatten('F')))

    L3_response_low_all = np.concatenate((L3_response_low_mixed.flatten('F'), L3_response_low_mixed.flatten('F')))
    
    fitx = np.transpose(np.stack((L2_response_all, L3_active_all, L3_active_all*filter_high_all, \
                       L3_active_all*L3_response_high_all, L3_active_all*L3_response_low_all, \
                      L3_active_all*np.log10(lum_edge_all)*np.tanh(lum_edge_all))))
    
    idx_train_2 = np.intersect1d(idx_train, np.where(L3_active_all == 0))
    
    reg = LinearRegression().fit(fitx[idx_train_2, :], real_response_all[idx_train_2])
    
    return reg.score(fitx[idx_train_2, :], real_response_all[idx_train_2])

Use randomly chosen training set

In [11]:
iter_num = 30
test_ratio = 0.5

In [12]:
k0_rand = np.zeros(iter_num)
Rsq_rand = np.zeros(iter_num)
k0_test = np.linspace(1., 3., 201)
Rsq_rand_all = np.zeros((len(k0_test), iter_num))
idx_test_rand = [[] for i in range(iter_num)]
for i in range(iter_num):
    idx_test_within_group = np.random.permutation(30)[:int(np.round(test_ratio*30))]
    idx_test = np.concatenate([np.where(L3_active_all == 0)[0][idx_test_within_group.astype(int)], \
        np.where(L3_active_all == 1)[0][idx_test_within_group.astype(int)]])
    idx_test_rand[i] = idx_test
    for j in range(len(k0_test)):
        k0 = np.min(k0_test) + j*(np.max(k0_test) - np.min(k0_test))/(len(k0_test) - 1)
        Rsq_rand_all[j, i] = fitting_mixed([k0, idx_test])
    k0_rand[i] = k0_test[Rsq_rand_all[:, i] == max(Rsq_rand_all[:, i])]

In [13]:
assert np.max(k0_rand) < np.max(k0_test) and np.min(k0_rand) > np.min(k0_test)

In [14]:
k0_rand

array([1.85, 2.07, 2.18, 2.13, 1.83, 1.62, 1.94, 1.79, 2.18, 2.1 , 2.25,
       2.19, 2.51, 1.89, 2.11, 2.49, 1.5 , 1.91, 1.92, 2.09, 2.1 , 2.06,
       1.94, 2.03, 1.87, 1.68, 2.39, 1.76, 2.06, 1.61])

In [15]:
scipy.io.savemat('bootstrap_mixed.mat', {'k0_rand': k0_rand, 'idx_test_rand': idx_test_rand})